<a href="https://colab.research.google.com/github/nidhi-059/Region-captioning-on-images-using-dl/blob/main/Task5_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.model_selection import train_test_split
from collections import Counter
import re


PAD_TOKEN = "<PAD>"
UNK_TOKEN = "<UNK>"
MAX_LEN = 200
VOCAB_SIZE = 10000
BATCH_SIZE = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


df = pd.read_csv("/content/IMDB Dataset.csv")
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['review'], df['sentiment'], test_size=0.2, random_state=42
)

class IMDBDataset(Dataset):
    def __init__(self, texts, labels, vocab=None):
        self.texts = [self.tokenize(t) for t in texts]
        self.labels = [1 if label == 'positive' else 0 for label in labels]

        if vocab is None:
            self.vocab = self.build_vocab(self.texts)
        else:
            self.vocab = vocab

        self.texts = [self.encode(t) for t in self.texts]

    def tokenize(self, text):
        text = text.lower()
        text = re.sub(r"<.*?>", "", text)
        text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
        return text.split()

    def build_vocab(self, tokenized_texts):
        counter = Counter(token for tokens in tokenized_texts for token in tokens)
        most_common = counter.most_common(VOCAB_SIZE - 2)
        vocab = {PAD_TOKEN: 0, UNK_TOKEN: 1}
        vocab.update({word: i+2 for i, (word, _) in enumerate(most_common)})
        return vocab

    def encode(self, tokens):
        ids = [self.vocab.get(token, self.vocab[UNK_TOKEN]) for token in tokens]
        ids = ids[:MAX_LEN]
        return ids + [self.vocab[PAD_TOKEN]] * (MAX_LEN - len(ids))

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.texts[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.float)

train_dataset = IMDBDataset(train_texts.tolist(), train_labels.tolist())
test_dataset = IMDBDataset(test_texts.tolist(), test_labels.tolist(), vocab=train_dataset.vocab)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
vocab_size = len(train_dataset.vocab)

print(" Data loaded | Vocab size:", vocab_size)




 Data loaded | Vocab size: 10000


In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, _) = self.lstm(embedded)
        return self.sigmoid(self.fc(hidden[-1]))

model = LSTMClassifier(vocab_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

def train_model(model):
    model.train()
    for epoch in range(10):
        total_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            preds = model(inputs).squeeze(1)
            loss = criterion(preds, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1} | Loss: {total_loss/len(train_loader):.4f}")

def evaluate_model(model):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            preds = model(inputs).squeeze(1)
            predicted = (preds >= 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    print(f"LSTM Accuracy: {100 * correct / total:.2f}%")

train_model(model)
evaluate_model(model)


Epoch 1 | Loss: 0.6902
Epoch 2 | Loss: 0.6631
Epoch 3 | Loss: 0.3717
Epoch 4 | Loss: 0.2653
Epoch 5 | Loss: 0.2103
Epoch 6 | Loss: 0.1621
Epoch 7 | Loss: 0.1167
Epoch 8 | Loss: 0.0809
Epoch 9 | Loss: 0.0592
Epoch 10 | Loss: 0.0457
LSTM Accuracy: 86.53%
